In [ ]:
from pathlib import Path
import struct
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
FILE_PATH = Path("./asd_data/1096.asd")

In [ ]:
def read_uint8(file):
    result = int.from_bytes(file.read(1), byteorder="little")
    return result


def read_int8(file):
    result = struct.unpack("b", file.read(1))[0]
    return result


def read_int16(file):
    result = struct.unpack("h", file.read(2))[0]
    return result


def read_int32(file):
    result = struct.unpack("i", file.read(4))[0]
    return result


def read_uint32(file):
    result = struct.unpack("<I", file.read(4))[0]
    return result


def read_hex_u32(file):
    result = hex(struct.unpack("<I", file.read(4))[0])
    return result


def read_float(file):
    result = struct.unpack("f", file.read(4))[0]
    return result


def read_bool(file):
    result = bool(int.from_bytes(file.read(1), byteorder="little"))
    return result


def read_ascii(file, length_bytes=1):
    bytes = file.read(length_bytes)
    string = bytes.decode("ascii")
    return string


def skip_bytes(file, length_bytes=1):
    skipped_bytes = file.read(length_bytes)
    return skipped_bytes

In [ ]:
# Channel encoding all in little endian
# topography: 0x5054 decodes to 'TP' in ascii little endian
# error: 0x5245 decodes to 'ER' in ascii little endian
# phase: 0x4850 decodes to 'PH' in ascii little endian
# none: 0x0000 no channel.

In [ ]:
# Read file as binary
with open(FILE_PATH, "rb") as f:
    # ==== HEADER ====
    # File version?
    unknown_int = read_int32(f)
    # channels available
    channel1 = read_ascii(f, 2)
    channel2 = read_ascii(f, 2)
    # # length of header in bytes
    header_length = read_int32(f)
    # I do not know what these are
    frame_header_length = read_int32(f)
    operator_name_size = read_int32(f)
    comment_offset_size = read_int32(f)
    comment_size = read_int32(f)
    # x and y resolution (pixels)
    x_pixels = read_int16(f)
    y_pixels = read_int16(f)
    # x and y resolution (nm)
    x_nm = read_int16(f)
    y_nm = read_int16(f)
    # frame time
    frame_rate = read_float(f)
    # z piezo extension
    z_piezo_extension = read_float(f)
    # z piezo gain
    z_piezo_gain = read_float(f)
    # No idea what AD range is
    ad_range = read_hex_u32(f)
    # Number of bits of data
    data_bits_size = read_int32(f)
    # Not sure, something to do with data averaging
    is_averaged = read_bool(f)
    # Window for averaging the data
    averaging_window = read_int32(f)
    # Some padding to ensure backwards compatilibilty I think
    _ = read_int16(f)
    # Date of creation
    year = read_int16(f)
    month = read_uint8(f)
    day = read_uint8(f)
    hour = read_uint8(f)
    minute = read_uint8(f)
    second = read_uint8(f)
    # Rounding degree?
    rounding_degree = read_uint8(f)
    # Maximum x and y scanning range in real space, nm
    max_x_scan_range = read_float(f)
    max_y_scan_range = read_float(f)
    # No idea
    _ = read_int32(f)
    _ = read_int32(f)
    _ = read_int32(f)
    # Number of frames the file had when recorded
    initial_frames = read_int32(f)
    # Actual number of frames
    num_frames = read_int32(f)
    # ID of the AFM instrument
    afm_ID = read_int32(f)
    # ID of the file
    file_id = read_int16(f)
    # Name of the user
    user_name = read_ascii(f, length_bytes=operator_name_size)
    # Sensitivity of the scanner in nm / V
    scanner_sensitivity = read_float(f)
    # Phase sensitivity
    phase_sensitivity = read_float(f)
    # Direction of the scan
    scan_direction = read_int32(f)
    # Skip bytes: comment offset size
    _ = skip_bytes(f, comment_offset_size)
    # Read a comment
    comment = []
    print(f"reading comment of length {comment_size}")
    for i in range(comment_size):
        comment.append(chr(read_int8(f)))
    print(comment)
    comment_without_null = "".join([c for c in comment if c != "\x00"])
    print(f"comment with null values removed: {comment_without_null}")

    # Read the actual data
    print(f"---")

    frames = []

    for i in range(num_frames):
        frame_number = read_int32(f)
        print(f"frame number: {frame_number}")
        max_data = read_int16(f)
        print(f"max data: {max_data}")
        min_data = read_int16(f)
        print(f"min data: {min_data}")
        x_offset = read_int16(f)
        print(f"x offset: {x_offset}")
        y_offset = read_int16(f)
        print(f"y offset: {y_offset}")
        x_tilt = read_float(f)
        print(f"x tilt: {x_tilt}")
        y_tilt = read_float(f)
        print(f"y tilt: {y_tilt}")
        is_stimulated = read_bool(f)
        print(f"is stimulated: {is_stimulated}")
        _booked_1 = read_int8(f)
        _booked_2 = read_int16(f)
        _booked_3 = read_int32(f)
        _booked_4 = read_int32(f)

        total_size = x_pixels * y_pixels
        # Each
        total_byte_size = total_size * 2
        frame_data = f.read(total_size * 2)

        frame_data = np.frombuffer(frame_data, dtype=np.int16)

        frame_data = frame_data.reshape((y_pixels, x_pixels))

        frames.append(frame_data)

In [ ]:
fig = plt.figure()


def update(i):
    plt.clf()  # clear the current frame
    plt.imshow(frames[i], cmap="gray")


ani = animation.FuncAnimation(fig, update, frames=len(frames))